In [1]:

import os
import glob
from tifffile import imread, imwrite
import numpy as np
from tqdm import tqdm
from vollseg import Augmentation3D
from pathlib import Path
from scipy.ndimage import gaussian_filter

PyDIPjavaio unavailable:
DLL load failed while importing PyDIPjavaio: The specified module could not be found.

libjvm not found
DIPlib -- a quantitative image analysis library
Version 3.1.0 (Sep 24 2021)
For more information see https://diplib.org


In [2]:
image_dir =  '/raw_dir/'
label_dir = '/label_dir/'

Aug_image_dir =  '/aug_raw_dir/'
Aug_label_dir = '/aug_label_dir/'

Path(Aug_image_dir).mkdir(exist_ok=True)
Path(Aug_label_dir).mkdir(exist_ok=True)


In [3]:
#All choices below are optional
size = [40,800,800]
gauss_filter_size = 0
#choices for augmentation below are 1 or 2 or None
flip_axis= 1
shift_axis= 1
zoom_axis= 1
#shift range can be between -1 and 1 (-1 and 1 will translate the pixels completely out), zoom range > 0
shift_range= 0.2 
zoom_range= 2
rotate_axis= 1
rotate_angle= 'random'
#if zero padding also needs to be done
size_zero = [1000,1000]
#Add poisson noise to data with mu
mu = 5

In [4]:
Raw_path = os.path.join(image_dir, '*tif')
filesRaw = glob.glob(Raw_path)
filesRaw.sort

Label_path = os.path.join(label_dir, '*tif')
filesLabel = glob.glob(Label_path)
filesLabel.sort

Data = []
Label = []
for fname in filesRaw:

    for secondfname in filesLabel:

        Name = os.path.basename(os.path.splitext(fname)[0])
        LabelName = os.path.basename(os.path.splitext(secondfname)[0])
        if Name == LabelName:
                image = imread(fname)
                if image.shape == (38,512,512):
                    Data.append(image)
                    labelimage = gaussian_filter(imread(secondfname), gauss_filter_size)
                    Label.append(labelimage)   
            
Data = np.asarray(Data)
Label = np.asarray(Label)


First perform the extend pixels to a certain size and zero padding operations

In [ ]:
extend_pixels = Augmentation3D(size=size)
aug_extend_pixels = extend_pixels.build(data=Data, label=Label, batch_size = Data.shape[0])
aug_extend_pixels_pair = np.asarray(next(aug_extend_pixels))
count = 0
for i in range(0, aug_extend_pixels_pair.shape[1]):
     Name = 'aug_extend_pixels' + str(count)
     imwrite(Aug_image_dir + '/' + Name + '.tif', aug_extend_pixels_pair[0,i,:,:].astype('float32'))
     imwrite(Aug_label_dir + '/' + Name + '.tif', aug_extend_pixels_pair[1,i,:,:].astype('uint16'))
     count = count + 1   

In [5]:
embed_pixels = Augmentation3D(size_zero=size_zero)
aug_embed_pixels = embed_pixels.build(data=Data, label=Label, batch_size = Data.shape[0])
aug_embed_pixels_pair = np.asarray(next(aug_embed_pixels))
count = 0
for i in range(0, aug_embed_pixels_pair.shape[1]):
     Name = 'aug_embed_pixels' + str(count)
     imwrite(Aug_image_dir + '/' + Name + '.tif', aug_embed_pixels_pair[0,i,:,:].astype('float32'))
     imwrite(Aug_label_dir + '/' + Name + '.tif', aug_embed_pixels_pair[1,i,:,:].astype('uint16'))
     count = count + 1   

In [ ]:
rotate_pixels = Augmentation3D(rotate_axis = rotate_axis, rotate_angle = rotate_angle)
aug_rotate_pixels = rotate_pixels.build(data=Data, label=Label, batch_size = Data.shape[0])
aug_rotate_pixels_pair = np.asarray(next(aug_rotate_pixels))
count = 0
for i in range(0, aug_rotate_pixels_pair.shape[1]):
     Name = 'rotate_pixels' + str(count)
     imwrite(Aug_image_dir + '/' + Name + '.tif', aug_rotate_pixels_pair[0,i,:,:].astype('float32'))
     imwrite(Aug_label_dir + '/' + Name + '.tif', aug_rotate_pixels_pair[1,i,:,:].astype('uint16'))
     count = count + 1   

In [ ]:
flip_pixels = Augmentation3D(flip_axis = flip_axis)
aug_flip_pixels = flip_pixels.build(data=Data, label=Label, batch_size = Data.shape[0])
aug_flip_pixels_pair = np.asarray(next(aug_flip_pixels))
count = 0
for i in range(0, aug_flip_pixels_pair.shape[1]):
     Name = 'aug_flip_pixels' + str(count)
     imwrite(Aug_image_dir + '/' + Name + '.tif', aug_flip_pixels_pair[0,i,:,:].astype('float32'))
     imwrite(Aug_label_dir + '/' + Name + '.tif', aug_flip_pixels_pair[1,i,:,:].astype('uint16'))
     count = count + 1   

Then Zoom and Shift augmentation

In [ ]:
zoom_pixels = Augmentation3D(zoom_axis = zoom_axis, zoom_range = zoom_range)
aug_zoom_pixels = zoom_pixels.build(data=Data, label=Label, batch_size = Data.shape[0])
aug_zoom_pixels_pair = np.asarray(next(aug_zoom_pixels))
count = 0
for i in range(0, aug_zoom_pixels_pair.shape[1]):
     Name = 'aug_zoom_pixels' + str(count)
     imwrite(Aug_image_dir + '/' + Name + '.tif', aug_zoom_pixels_pair[0,i,:,:].astype('float32'))
     imwrite(Aug_label_dir + '/' + Name + '.tif', aug_zoom_pixels_pair[1,i,:,:].astype('uint16'))
     count = count + 1   

In [ ]:
shift_pixels = Augmentation3D(shift_axis = shift_axis, shift_range = shift_range)
aug_shift_pixels = shift_pixels.build(data=Data, label=Label, batch_size = Data.shape[0])
aug_shift_pixels_pair = np.asarray(next(aug_shift_pixels))
count = 0
for i in range(0, aug_shift_pixels_pair.shape[1]):
     Name = 'aug_shift_pixels' + str(count)
     imwrite(Aug_image_dir + '/' + Name + '.tif', aug_shift_pixels_pair[0,i,:,:].astype('float32'))
     imwrite(Aug_label_dir + '/' + Name + '.tif', aug_shift_pixels_pair[1,i,:,:].astype('uint16'))
     count = count + 1   

Finally add noise to all the data generated so far

In [ ]:
Raw_path = os.path.join(Aug_image_dir, '*tif')
filesRaw = glob.glob(Raw_path)
filesRaw.sort

Label_path = os.path.join(Aug_label_dir, '*tif')
filesLabel = glob.glob(Label_path)
filesLabel.sort

Data = []
Label = []
for fname in filesRaw:
    
    for secondfname in filesLabel:

        Name = os.path.basename(os.path.splitext(fname)[0])
        LabelName = os.path.basename(os.path.splitext(secondfname)[0])
        if Name == LabelName:
                image = imread(fname)
                Data.append(image)
                labelimage = imread(secondfname)
                Label.append(labelimage)   
            
Data = np.asarray(Data)
Label = np.asarray(Label)


In [ ]:
noise_pixels = Augmentation3D(mu = mu)
aug_noise_pixels = noise_pixels.build(data=Data, label=Label, batch_size = Data.shape[0])
aug_noise_pixels_pair = np.asarray(next(aug_noise_pixels))
count = 0
for i in range(0, aug_noise_pixels_pair.shape[1]):
     Name = 'aug_noise_pixels' + str(count)
     imwrite(Aug_image_dir + '/' + Name + '.tif', aug_noise_pixels_pair[0,i,:,:].astype('float32'))
     imwrite(Aug_label_dir + '/' + Name + '.tif', aug_noise_pixels_pair[1,i,:,:].astype('uint16'))
     count = count + 1   